**1 Set up initial network**

In [1]:
import torch
from difflogic import LogicLayer, GroupSum
from chop import MaseGraph
from chop.passes import *
import mnist_dataset


# setup basic network

model = torch.nn.Sequential(
    torch.nn.Flatten(),
    LogicLayer(400, 8_000),
    LogicLayer(8_000, 8_000),
    LogicLayer(8_000, 8_000),
    LogicLayer(8_000, 8_000),
    LogicLayer(8_000, 8_000),
    LogicLayer(8_000, 8_000),
    GroupSum(k=10, tau=20)
)
model.eval()

CUSTOM_OPS = {
    "modules": {
        LogicLayer: {
            "args": {
                "input": "data_in"
            },
            "toolchain": "INTERNAL_RTL",
            "module": "fixed_difflogic_logic",
            "dependence_files": [
                "difflogic_layers/rtl/fixed_difflogic_logic.sv",
                "difflogic_layers/rtl/fixed_difflogic_logic_neuron.sv",
            ],
        },
        GroupSum: {
            "args": {
                "input": "data_in"
            },
            "toolchain": "INTERNAL_RTL",
            "module": "fixed_difflogic_groupsum",
            "dependence_files": [
                "difflogic_layers/rtl/fixed_difflogic_groupsum.sv",
            ],
        },
    },
    "functions": {},
}

mg = MaseGraph(model, custom_ops=CUSTOM_OPS)
mg.draw("out-svg/dlg_init.svg")


# load weights

state_dict = torch.load("best_model_full.pth", map_location=torch.device('cpu'))
mg.model.load_state_dict(state_dict, strict=False)


# add input data and analyse

train_set = mnist_dataset.MNIST('./data-mnist', train=True, download=True, remove_border=True)
test_set = mnist_dataset.MNIST('./data-mnist', train=False, remove_border=True)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True, pin_memory=True, drop_last=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=100, shuffle=False, pin_memory=True, drop_last=True)
dummy_in_loader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True, drop_last=True)
image, label = next(iter(dummy_in_loader))
image = image.cpu().squeeze(0) 

mg, _ = init_metadata_analysis_pass(mg)
mg, _ = add_common_metadata_analysis_pass(mg, pass_args={"dummy_in": {"input_1": image} })
mg.draw("out-svg/dlg_post_common.svg")

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


**2 Add additional hardware metadata hints**

In [2]:
from collections import OrderedDict

def difflogic_hardware_metadata_optimize_pass(graph, args={}):
    
    def _is_logiclayer(node):
        return node.meta["mase"]["common"]["mase_op"] == "user_defined_module"
    
    for node in graph.nodes:
        if _is_logiclayer(node):
            pre_common_args_md = node.meta["mase"]["common"]["args"]
            post_common_args_md = {}
            node.meta["mase"]["hardware"]["difflogic_args"] = {}
            for k, v in pre_common_args_md.items():
                if "data_in" not in k:
                    node.meta["mase"]["hardware"]["difflogic_args"][k] = v
                else:
                    post_common_args_md[k] = v
            post_common_args_md = OrderedDict(post_common_args_md)
            node.meta["mase"]["common"]["args"] = post_common_args_md
    return (graph, None)

In [3]:
from chop.passes.graph.analysis.add_metadata.add_hardware_metadata import *

def difflogic_hardware_force_fixed_flatten_pass(graph, args={}):
    for node in graph.nodes:
        if node.meta["mase"]["common"]["mase_op"] == "flatten":
            # add_component source
            node.meta["mase"]["hardware"]["toolchain"] = "INTERNAL_RTL"
            node.meta["mase"]["hardware"]["module"] = "fixed_difflogic_flatten"
            node.meta["mase"]["hardware"]["dependence_files"] = ["difflogic_layers/rtl/fixed_difflogic_flatten.sv"]
            # else
            add_verilog_param(node)
            add_extra_verilog_param(node, graph)
            graph.meta["mase"]["hardware"]["verilog_sources"] += node.meta["mase"]["hardware"]["dependence_files"]
    return (graph, None)

In [4]:
from collections import OrderedDict

def difflogic_annotate_io_nodes(graph, args={}):
    
    def _is_io_node(node):
        return node.op in ["placeholder", "output"]
    
    for node in graph.nodes:
        if _is_io_node(node):
            node.meta["mase"]["hardware"]["difflogic"] = True
    return (graph, None)

In [5]:
def print_metadata_pass(graph, args={}):
    if len(args) != 0:
        for node in graph.nodes:
            print(node.op)
            for k, v in node.meta["mase"][args["metadata_key"]].items():
                print(f"{k}: {v}")
            print()
    else:
        for node in graph.nodes:
            print(node.op)
            for k, v in node.meta["mase"]["common"].items():
                print(f"{k}: {v}")
            print()
    return mg, None

**3 Run metadata passes and generate hardware**

In [6]:
mg, _ = difflogic_hardware_metadata_optimize_pass(mg)

In [7]:
mg, _ = difflogic_annotate_io_nodes(mg)

In [8]:
mg, _ = add_hardware_metadata_analysis_pass(mg)

In [9]:
mg, _ = difflogic_hardware_force_fixed_flatten_pass(mg)

In [10]:
mg, _ = emit_verilog_top_transform_pass(mg)
mg, _ = emit_internal_rtl_transform_pass(mg)
mg, _ = emit_bram_transform_pass(mg)

**4 Generate cocotb testbench**

In [11]:
mg, _ = mg, _ = emit_cocotb_transform_pass(mg)

In [12]:
from chop.actions import simulate
simulate(skip_build=False, skip_test=False, build_jobs=8, unroll_count=10_000)

INFO: Running command perl /usr/local/bin/verilator -cc --exe -Mdir /workspace/mase-DLG/docs/DLG/sim_build -DCOCOTB_SIM=1 --top-module top --vpi --public-flat-rw --prefix Vtop -o top -LDFLAGS '-Wl,-rpath,/usr/local/lib/python3.11/dist-packages/cocotb/libs -L/usr/local/lib/python3.11/dist-packages/cocotb/libs -lcocotbvpi_verilator' -Wno-fatal -Wno-lint -Wno-style --trace-fst --trace-structs --trace-depth '--build-jobs 8' '--unroll-count 10000' 3 -I/root/.mase/top/hardware/rtl -I/workspace/mase-DLG/src/mase_components/interface/rtl -I/workspace/mase-DLG/src/mase_components/language_models/rtl -I/workspace/mase-DLG/src/mase_components/memory/rtl -I/workspace/mase-DLG/src/mase_components/vivado/rtl -I/workspace/mase-DLG/src/mase_components/convolution_layers/rtl -I/workspace/mase-DLG/src/mase_components/cast/rtl -I/workspace/mase-DLG/src/mase_components/systolic_arrays/rtl -I/workspace/mase-DLG/src/mase_components/scalar_operators/rtl -I/workspace/mase-DLG/src/mase_components/transformer_l

%Error: Invalid option: --unroll-count 10000


SystemExit: Process 'perl' terminated with error 1